This notebook presents a streamlined version of running the models in batches with updates on parameters.

In [1]:
%run nlp-functions.ipynb

2024-05-22 01:13:54.192204: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-05-22 01:13:54.192228: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-05-22 01:13:54.192231: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-05-22 01:13:54.192248: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-22 01:13:54.192257: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
df1 = pd.read_json('initial_predictions.json')

In [5]:
params = {'embed_dim' : [24, 32],
          'ff_dim' : [48, 64,96],
          'Batch Size' : 5,
          'Num Transformer Heads' : [1, 2, 4, 8],
          'Sample Random State': 72}

NUM_NEG_SAMPLES = 1500
TEST_SIZE = 0.2
results_df = RecordDataFrame()
import itertools
FILE = 'lstm.csv'
RESULTS = 'individual_layers.csv'

dim_combos = set(list(itertools.combinations(itertools.chain(params['embed_dim'], params['ff_dim']),2)))
results_list = []

dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, params['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']

In [6]:
batch = 16
train_dataset, val_dataset = get_training_data(X,y,state = params['Sample Random State'], batch_size = batch)
num = 1

for (embed_dim, ff_dim) in list(dim_combos):
    for i in range(3):

        r = {'Model Description' : 'NER Transformer With One LSTM',
                'Sample Random State' : params['Sample Random State'],
                 'Epochs' : 10,
                 'embed_dim' : embed_dim,
                 'Num Transformer Heads' : num,
                 'ff_dim' : ff_dim,
                 'Batch Size' : batch}

        print(r)
        early_stopping = tf.keras.callbacks.EarlyStopping(mode = 'min', patience = 2)
        ner_model = NERModelWithOneLSTM(num_tags, 
                                              vocab_size, 
                                              embed_dim=r['embed_dim'], 
                                              num_heads=r['Num Transformer Heads'], 
                                              ff_dim=r['ff_dim'])
        ner_model.compile(optimizer='adam',loss=loss)
        ner_model.fit(train_dataset, epochs=r['Epochs'],validation_data = val_dataset,callbacks = early_stopping)
        tmp_results, record_dict = calculate_metrics(val_dataset, ner_model)
        r.update({'Dataset Type': 'validation'})
        r.update(record_dict)
        results_df.add_record(r)
        results_df.to_csv(FILE)
        results_list.append(tmp_results)
        pd.concat(results_list,axis = 0,ignore_index=True).to_csv(RESULTS)
        micro_f5 = record_dict['Micro F5']
        if micro_f5 > 0.7:
            test_dataset = get_test_data(train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)), 'tokens'],
                             train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)),'labels'], batch)
            tmp2_results, record_dict = calculate_metrics(test_dataset, ner_model)
            r.update({'Dataset Type': 'test'})
            r.update(record_dict)
            results_df.add_record(r)
            results_df.to_csv(FILE)
            results_list.append(tmp2_results)
            pd.concat(results_list,axis = 0,ignore_index=True).to_csv(RESULTs)

{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 24, 'Num Transformer Heads': 1, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


2024-05-21 21:56:39.248238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


127/127 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - loss: 0.7831 - val_loss: 0.1846
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.1870 - val_loss: 0.1734
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 0.1297 - val_loss: 0.1620
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0426 - val_loss: 0.1752
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.0182 - val_loss: 0.1797
processed 9843 tokens
True Positive: 151, False Positive: 53, False Negative: 209
Weighted Precision: 0.725844280240832
Weighted Recall: 0.4194444444444444
Weighted F1: 0.5184263424382733
Micro F5: 0.4265536723163842
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 24, 'Num Transformer Heads': 1, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


2024-05-21 21:57:16.092017: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 36s 58ms/step - loss: 0.5336 - val_loss: 0.1688
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.1375 - val_loss: 0.1331
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.0514 - val_loss: 0.0831
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.0198 - val_loss: 0.0823
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - loss: 0.0098 - val_loss: 0.0924
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: 0.0066 - val_loss: 0.1116
processed 9843 tokens
True Positive: 167, False Positive: 68, False Negative: 193
Weighted Precision: 0.8273875007230029
Weighted Recall: 0.46388888888888885
Weighted F1: 0.589777198997629
Micro F5: 0.4701678397401191
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 24, 'Num Transformer Heads': 1, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


...

2024-05-21 22:25:39.994486: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 52s 304ms/step - loss: 0.7275 - val_loss: 0.1732
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - loss: 0.1373 - val_loss: 0.1384
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - loss: 0.0504 - val_loss: 0.1198
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: 0.0222 - val_loss: 0.1311
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - loss: 0.0112 - val_loss: 0.1613
processed 9843 tokens
True Positive: 134, False Positive: 43, False Negative: 226
Weighted Precision: 0.8051892995570321
Weighted Recall: 0.37222222222222223
Weighted F1: 0.4938621449281574
Micro F5: 0.37964476408412334
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 32, 'Num Transformer Heads': 1, 'ff_dim': 48, 'Batch Size': 16}
Epoch 1/10


2024-05-21 22:27:06.839634: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 52s 315ms/step - loss: 0.7175 - val_loss: 0.1735
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - loss: 0.1478 - val_loss: 0.1492
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - loss: 0.0534 - val_loss: 0.1147
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - loss: 0.0229 - val_loss: 0.1537
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - loss: 0.0112 - val_loss: 0.1435
processed 9843 tokens
True Positive: 145, False Positive: 69, False Negative: 215
Weighted Precision: 0.6495169082125604
Weighted Recall: 0.4027777777777778
Weighted F1: 0.4894119937694704
Micro F5: 0.4091599739526807


2024-05-21 22:28:38.801278: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
batch = 16
train_dataset, val_dataset = get_training_data(X,y,state = params['Sample Random State'], batch_size = batch)

for num in params['Num Transformer Heads']:
    for (embed_dim, ff_dim) in list(dim_combos):
        for i in range(3):

            r = {'Model Description' : 'NER Transformer With One LSTM',
                 'Sample Random State' : params['Sample Random State'],
                 'Epochs' : 5,
                 'embed_dim' : embed_dim,
                 'Num Transformer Heads' : num,
                 'ff_dim' : ff_dim,
                 'Batch Size' : batch}

            print(r)
            ner_model = NERModelWithOneLSTM(num_tags, 
                                              vocab_size, 
                                              embed_dim=r['embed_dim'], 
                                              num_heads=r['Num Transformer Heads'], 
                                              ff_dim=r['ff_dim'])
            ner_model.compile(optimizer='adam',loss=loss)
            ner_model.fit(train_dataset, epochs=r['Epochs'])
            tmp_results, record_dict = calculate_metrics(val_dataset, ner_model)
            r.update({'Dataset Type': 'validation'})
            r.update(record_dict)
            results_df.add_record(r)
            results_df.to_csv(FILE)
            results_list.append(tmp_results)
            pd.concat(results_list,axis = 0,ignore_index=True).to_csv(RESULTS)
            micro_f5 = record_dict['Micro F5']
            if micro_f5 > 0.7:
                test_dataset = get_test_data(train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)), 'tokens'],
                             train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)),'labels'], batch)
                tmp2_results, record_dict = calculate_metrics(test_dataset, ner_model)
                r.update({'Dataset Type': 'test'})
                r.update(record_dict)
                results_df.add_record(r)
                results_df.to_csv('lstm.csv')
                results_list.append(tmp2_results)
                pd.concat(results_list,axis = 0,ignore_index=True).to_csv(RESULTS)

{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 5, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 43s 230ms/step - loss: 0.5676
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.1215
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 0.0468
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.0181
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 0.0099
processed 9843 tokens
True Positive: 229, False Positive: 132, False Negative: 131
Weighted Precision: 0.6709186322504079
Weighted Recall: 0.6361111111111111
Weighted F1: 0.6434298941870492
Micro F5: 0.6360431577822883
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 5, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/5


2024-05-21 11:10:40.015520: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 44s 252ms/step - loss: 0.6903
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 0.1654
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.0578
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.0210
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.0101
processed 9843 tokens
True Positive: 180, False Positive: 97, False Negative: 180
Weighted Precision: 0.6760855094078886
Weighted Recall: 0.5
Weighted F1: 0.5619142437263984
Micro F5: 0.5044734289102081
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 5, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/5


2024-05-21 11:11:54.863017: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 45s 252ms/step - loss: 0.7521
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - loss: 0.1462
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 0.0561
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 0.0207
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: 0.0098
processed 9843 tokens
True Positive: 200, False Positive: 107, False Negative: 160
Weighted Precision: 0.7687041138591526
Weighted Recall: 0.5555555555555555
Weighted F1: 0.6142327194355598
Micro F5: 0.5587192435801009
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 5, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 64, 'Batch Size': 16}
Epoch 1/5


...

127/127 ━━━━━━━━━━━━━━━━━━━━ 61s 367ms/step - loss: 0.5810
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 62ms/step - loss: 0.1963
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - loss: 0.1777
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - loss: 0.0980
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - loss: 0.0645
processed 9843 tokens
True Positive: 121, False Positive: 62, False Negative: 239
Weighted Precision: 0.5983478471214321
Weighted Recall: 0.3361111111111111
Weighted F1: 0.4205859957401491
Micro F5: 0.3425895676794076
{'Model Description': 'NER Transformer With One LSTM', 'Sample Random State': 72, 'Epochs': 5, 'embed_dim': 48, 'Num Transformer Heads': 4, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/5


2024-05-21 12:22:48.981116: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 70s 387ms/step - loss: 0.5060
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: 0.1382
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - loss: 0.0394
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - loss: 0.0176
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: 0.0078


2024-05-21 12:24:34.730260: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


processed 9843 tokens
True Positive: 278, False Positive: 126, False Negative: 82
Weighted Precision: 0.6950490196078432
Weighted Recall: 0.7722222222222221
Weighted F1: 0.7249889381220358
Micro F5: 0.7686091025095704
